In [99]:
import pandas as pd
import numpy as np
import re

import random
from nltk.stem.porter import PorterStemmer

In [34]:
anime_d=pd.read_csv("./data/anime/anime_2022.csv")

In [26]:
new_ani = anime_d.drop(['ID',
 
 'Synonyms',
 'Japanese',
 'Premiered',
 'Broadcast',
 'Licensors',
 'Type',
 'Episodes',
 'Status',
 'Start_Aired',
 'End_Aired',
 'Duration_Minutes',
 'Rating'
 ],axis=1)


In [28]:
# Removing 'Unknown'
#English
new_ani['English']=np.where(new_ani['English']=='Unknown',new_ani['Title'],new_ani['English'])
#Others
new_ani=new_ani.replace("Unknown", "")
                             
                             

In [33]:
new_ani=new_ani.drop(['Title'],axis=1)

In [43]:
new_ani.Source.value_counts()

Original        7324
Manga           4408
                3602
Visual novel    1093
Game            1059
Light novel      898
Other            849
Novel            634
Music            357
Web manga        352
4-koma manga     306
Picture book     184
Book             156
Mixed media      111
Card game         65
Web novel         49
Radio             13
Name: Source, dtype: int64

In [87]:
new_ani.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21460 entries, 0 to 21459
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   English            21460 non-null  object 
 1   Synopsis           21460 non-null  object 
 2   Producers          21460 non-null  object 
 3   Studios            21460 non-null  object 
 4   Source             21460 non-null  object 
 5   Genres             21460 non-null  object 
 6   Themes             21460 non-null  object 
 7   Demographics       21460 non-null  object 
 8   Score              14562 non-null  float64
 9   Scored_Users       14562 non-null  float64
 10  Ranked             19536 non-null  float64
 11  Popularity         21460 non-null  int64  
 12  Members            21460 non-null  int64  
 13  Favorites          21460 non-null  int64  
 14  Modified Synopsis  21460 non-null  object 
 15  Tags               21460 non-null  object 
dtypes: float64(3), int64(3

In [108]:
new_ani.head()

,English,Synopsis,Producers,Studios,Source,Genres,Themes,Demographics,Score,Scored_Users,Ranked,Popularity,Members,Favorites,Modified Synopsis,Tags
0,Attack on Titan,"Centuries ago, mankind was slaughtered to near...","Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,"Action, Drama","Gore, Military, Survival",Shounen,8.531,519803.0,1002.0,1,3524109,155695,centuri ago mankind wa slaughter to near extin...,centuri ago mankind wa slaughter to near extin...
1,Death Note,"Brutal murders, petty thefts, and senseless vi...","VAP, Konami, Ashi Productions, Nippon Televisi...",Madhouse,Manga,"Supernatural, Suspense",Psychological,Shounen,8.621,485487.0,732.0,2,3504535,159701,brutal murder petti theft and senseless violen...,brutal murder petti theft and senseless violen...
2,Fullmetal Alchemist Brotherhood,After a horrific alchemy experiment goes wrong...,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,"Action, Adventure, Drama, Fantasy",Military,Shounen,9.131,900398.0,12.0,3,2978455,207772,after a horrif alchemi experi goe wrong in the...,after a horrif alchemi experi goe wrong in the...
3,One Punch Man,The seemingly unimpressive Saitama has a rathe...,"TV Tokyo, Bandai Visual, Lantis, Asatsu DK, Ba...",Madhouse,Web manga,"Action, Comedy","Parody, Super Power",Seinen,8.511,19066.0,1112.0,4,2879907,59651,the seemingli unimpress saitama ha a rather un...,the seemingli unimpress saitama ha a rather un...
4,Sword Art Online,Ever since the release of the innovative Nerve...,"Aniplex, Genco, DAX Production, ASCII Media Wo...",A-1 Pictures,Light novel,"Action, Adventure, Fantasy, Romance","Love Polygon, Video Game",,7.201,990254.0,29562.0,5,2813565,64997,ever sinc the releas of the innov nervegear ga...,ever sinc the releas of the innov nervegear ga...


In [39]:
# synopsis alteration
ps = PorterStemmer()
def remove_splChar_normalizeWords(ss_line):
    word_list = []
    ss_line = re.sub('[^A-Za-z0-9]',' ', ss_line)
    #ss_line.replace('\n','')
    for word in ss_line.split():
        word_list.append(ps.stem(word))    
    return  " ".join(word_list)


new_ani['Modified Synopsis'] = new_ani['Synopsis'].apply(remove_splChar_normalizeWords)


In [47]:
#remove commas and creating new column
new_ani["Tags"] = new_ani['Modified Synopsis'] + " " + new_ani['Source'] + " " +[v.replace(',','') for v in new_ani['Producers']] + " " + [v.replace(',','') for v in new_ani['Studios']] + " " +[v.replace(',','') for v in new_ani['Genres']] + " " + [v.replace(',','') for v in new_ani['Themes']] + " " +[v.replace(',','') for v in new_ani['Demographics']]



In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

#getting tfidf
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
tfv_matrix = tfv.fit_transform(new_ani["Tags"])

In [58]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

#getting the indices of anime title
indices = pd.Series(new_ani.index, index=new_ani['English']).drop_duplicates()

In [90]:

def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    
    same_anime = [a for a in sig_scores if title in new_ani['English'].iloc[a[0]]]
    diff_anime = [a for a in sig_scores if a[0] not in [i for i,val in same_anime]]
    s_len=len(same_anime) if len(same_anime)<4 else 3 
    # Scores of the 10 most similar movies
    sig_scores = same_anime[1:s_len] + diff_anime[:(11-s_len)]

    # Movie indices
    anime_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return pd.DataFrame({'Anime name': new_ani['English'].iloc[anime_indices].values,
                                 'Rating': new_ani['Score'].iloc[anime_indices].values})


In [91]:
give_rec('And you thought there is never a girl online?')

,Anime name,Rating
0,Girlish Number,6.831
1,Ro-Kyu-Bu! SS,7.041
2,Angel's 3Piece!,6.621
3,Sword Art Online,7.201
4,Tawawa on Monday Specials,6.581
5,Suite Precure♪ Movie: Torimodose! Kokoro ga Ts...,7.001
6,Kiss x Sis (TV),6.631
7,Zero Escape VLR prologue,6.061
8,Shomin Sample,6.711
9,Gamers!,6.781


In [94]:
def getAllTitlesAvailable():
    return list(new_ani['English'])

In [102]:
#get rangom anime title and rationg from top ranked
def getRandomSuggestion(n):
    return random.sample(list(new_ani['English'])[:300], n)